In [1]:
import pymc as pm
from scipy import stats, optimize
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
N = [5, 20, 30]
MAX_ORDERS = [100, 80, 100]
PRICES = [16, 10, 12]

EFF_MU = [0.9, 0.5, 0.8]
EFF_SD = [0.1, 0.2, 0.2]
np.random.seed(100)
data = [pm.draw(pm.Beta.dist(mu=mu, sigma=sd, shape=n)) for mu, sd, n in zip(EFF_MU, EFF_SD, N)]
data

[array([0.95365031, 0.85029781, 0.81658076, 0.89917944, 0.97359409]),
 array([0.66254269, 0.83498257, 0.17640053, 0.2040686 , 0.53963025,
        0.56229829, 0.30094611, 0.35764109, 0.56655823, 0.6557655 ,
        0.56225169, 0.45198602, 0.47953829, 0.96759831, 0.45723895,
        0.44990676, 0.59351616, 0.38192819, 0.54552641, 0.29135157]),
 array([0.97655855, 0.98788624, 0.99375181, 0.63963936, 0.99997697,
        0.63357867, 0.59523406, 0.7161449 , 0.74003995, 0.96513338,
        0.7830355 , 0.99723442, 0.92078969, 0.77956119, 0.99215375,
        0.75524377, 0.27451075, 0.69827565, 0.95883063, 0.87567068,
        0.34627744, 0.9805346 , 0.87489206, 0.80403474, 0.99904852,
        0.98359068, 0.96186854, 0.98189893, 0.98591117, 0.91988753])]

In [3]:
SELL_PRICE = 40
HOLDING_COST = 10
BUY_PRICE = 10
MORE_STOCK = 70
LESS_STOCK = 30
DEMAND = 50
@np.vectorize
def loss(in_stock, demand, buy_price=BUY_PRICE, sales_price=SELL_PRICE, holding_cost=HOLDING_COST):
    margin = sales_price - buy_price
    if in_stock > demand:
        total_profit = demand * margin
        total_holding_cost = (in_stock - demand) * holding_cost
        reward = total_profit - total_holding_cost
    else:
        reward = in_stock * margin
    return -reward

a = np.arange(50)
b = np.arange(50)
y = loss(a, b)
y

array([    0,   -30,   -60,   -90,  -120,  -150,  -180,  -210,  -240,
        -270,  -300,  -330,  -360,  -390,  -420,  -450,  -480,  -510,
        -540,  -570,  -600,  -630,  -660,  -690,  -720,  -750,  -780,
        -810,  -840,  -870,  -900,  -930,  -960,  -990, -1020, -1050,
       -1080, -1110, -1140, -1170, -1200, -1230, -1260, -1290, -1320,
       -1350, -1380, -1410, -1440, -1470])

In [4]:
demand_sampled = stats.poisson(60, 40).rvs(1000)

In [10]:
with pm.Model() as model:
    alpha = pm.HalfNormal('alpha', sigma=10., shape=3) + 1
    beta = pm.HalfNormal('beta', sigma=10., shape=3) + 1
    
    for i, d in enumerate(data):
        pm.Beta('Supplier #' + str(i+1), alpha=alpha[i], beta=beta[i], observed=d)
    
    trace = pm.sample()
    pred = pm.sample_posterior_predictive(trace)
    
pm.summary(trace)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [alpha, beta]


Sampling 4 chains for 1_000 tune and 1_000 draw iterations (4_000 + 4_000 draws total) took 2 seconds.
Sampling: [Supplier #1, Supplier #2, Supplier #3]


,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
alpha[0],13.752,5.347,4.308,23.902,0.115,0.081,1983.0,1922.0,1.00
alpha[1],1.877,0.799,0.440,3.362,0.018,0.013,1808.0,1807.0,1.00
alpha[2],3.974,0.892,2.312,5.666,0.018,0.013,2433.0,2097.0,1.00
beta[0],0.926,0.658,0.005,2.083,0.014,0.010,1755.0,1433.0,1.00
beta[1],1.714,0.754,0.375,3.069,0.017,0.012,1790.0,1974.0,1.00
beta[2],0.040,0.038,0.000,0.113,0.001,0.000,2255.0,992.0,1.01


In [13]:
efficacy_scores = []
for supplier in ['Supplier #1','Supplier #2','Supplier #3']:
    samples = pred.posterior_predictive[supplier].values.flatten()
    efficacy_scores.append(np.random.choice(samples, size=1000, replace=False))
efficacy_scores = np.stack(efficacy_scores).T
efficacy_scores.shape

(1000, 3)

In [15]:
def calc_eff_and_price(orders, eff=np.array([.9, .5, .8]), prices=PRICES):
    orders = np.array(orders)
    full_eff = np.sum(eff * orders)
    unit_price = np.sum(orders * prices) / np.sum(orders)
    return full_eff, unit_price

def objective_func(orders, efficacy_scores=efficacy_scores, demand_sampled=demand_sampled, max_orders=MAX_ORDERS):
    orders = np.array(orders)
    losses = []
    for i in range(efficacy_scores.shape[0]):
        eff = efficacy_scores[i,:]
        eff, unit_price = calc_eff_and_price(orders, eff=eff)
        loss_i = loss(eff, demand_sampled[i], unit_price)
        losses.append(loss_i)
    return np.array(losses)

bounds = [(0, max_order) for max_order in MAX_ORDERS]
starting_value = [50, 50, 50]
opt_bayes = optimize.minimize(lambda *args: np.mean(objective_func(*args)), 
                              starting_value, 
                              bounds=bounds)
print('Number of orders per supplier', format(np.ceil(opt_bayes.x)))

Number of orders per supplier [ 0. 65. 92.]


In [21]:
efficacy_mean = np.array([[np.mean(d) for d in data]])
bounds = [(0, max_order) for max_order in MAX_ORDERS]
starting_value = [50, 50, 50]
opt_naive = optimize.minimize(lambda *args: objective_func(*args, efficacy_scores=efficacy_mean), 
                                  starting_value,
                                  bounds=bounds)
print('Optimal order amount from every supplier = {}'.format(np.ceil(opt_naive.x)))

Optimal order amount from every supplier = [44. 49. 46.]


In [31]:
np.random.seed(123)
data_new = []
for mu, sd, n in zip(EFF_MU, EFF_SD, N):
    data_new.append(pm.draw(pm.Beta.dist(mu=mu, sigma=sd, shape=1000)))
data_new = np.array(data_new).T
neg_loss_bayes = -objective_func(opt_bayes.x, efficacy_scores=data_new) / demand_sampled
neg_loss_naive = -objective_func(opt_naive.x, efficacy_scores=data_new) / demand_sampled
print('Profit of Bayesian Model:', np.mean(neg_loss_bayes), 'BTC')
print('Profit of Naive Model:', np.mean(neg_loss_naive), 'BTC')
print('Profit uplift:', (np.mean(neg_loss_bayes) - np.mean(neg_loss_naive)) / np.mean(neg_loss_naive))

Profit of Bayesian Model: 25.541053038506217 BTC
Profit of Naive Model: 25.067376700088914 BTC
Profit uplift: 0.01889612718891412
